# Libraries

In [1]:
# for data manipulation
import numpy as np
import pandas as pd

# our defined functions for data preparation
import utils.functions as ft

# setting option
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# for modeling
from sklearn.linear_model import LogisticRegression

# Load Data

In [2]:
pre = pd.read_csv("prepared_files/dp_previous.csv", index_col=0)
cre = pd.read_csv("prepared_files/dp_cre.csv", index_col=0)
pos = pd.read_csv("prepared_files/dp_pos.csv", index_col=0)
ins = pd.read_csv("prepared_files/dp_install.csv", index_col=0)
bu = pd.read_csv("prepared_files/dp_2bu.csv", index_col=0)
traintest = pd.read_csv("prepared_files/dp_traintest.csv", index_col=0)

# Data Preparation
__1. Merging All Dataset__

In [8]:
final = traintest.merge(pre.drop("TARGET", axis=1), on="SK_ID_CURR", how='left')
final = final.merge(cre.drop("TARGET", axis=1), on="SK_ID_CURR", how='left')
final = final.merge(pos.drop("TARGET", axis=1), on="SK_ID_CURR", how='left')
final = final.merge(ins.drop("TARGET", axis=1), on="SK_ID_CURR", how='left')
final = final.merge(bu.drop("TARGET", axis=1), on="SK_ID_CURR", how='left')

__2. Checking For Correlation__

In [9]:
# Checking correlation of all features with TARGET
pre_corr = pre.iloc[:, 1:].corr()['TARGET'].sort_values()
cre_corr = cre.iloc[:, 1:].corr()['TARGET'].sort_values()
pos_corr = pos.iloc[:, 1:].corr()['TARGET'].sort_values()
ins_corr = ins.iloc[:, 1:].corr()['TARGET'].sort_values()
bu_corr = bu.iloc[:, 1:].corr()['TARGET'].sort_values()
traintest_corr = traintest.iloc[:, 1:].corr()['TARGET'].sort_values()

__3. Remove Features Having Low Correlation With Target__

In [10]:
# Choosing features with absolute correlation lower than 0.001
final_drop_features = list(pre_corr[abs(pre_corr)<0.001].index)
final_drop_features += list(cre_corr[abs(cre_corr)<0.001].index)
final_drop_features += list(pos_corr[abs(pos_corr)<0.001].index)
final_drop_features += list(ins_corr[abs(ins_corr)<0.001].index)
final_drop_features += list(bu_corr[abs(bu_corr)<0.001].index)
final_drop_features += list(traintest_corr[abs(traintest_corr)<0.001].index)

In [11]:
# Remove features having low correlation with Target
final = final.drop(final_drop_features, axis=1)

__4. Polynomial Features__
<br>
Generate polynomial and interaction of those features having high correlation with TARGET (absolute values are more than 0.07)

In [12]:
# Choosing features with absolute correlation higher than 0.075 to generate polynomial and interaction
poly_features = list(pre_corr[abs(pre_corr)>0.075].index[:-1])
poly_features += list(cre_corr[abs(cre_corr)>0.075].index[:-1])
poly_features += list(pos_corr[abs(pos_corr)>0.075].index[:-1])
poly_features += list(ins_corr[abs(ins_corr)>0.075].index[:-1])
poly_features += list(bu_corr[abs(bu_corr)>0.075].index[:-1])
poly_features += list(traintest_corr[abs(traintest_corr)>0.075].index[:-1])

In [13]:
poly_features

['PRE_NAME_CONTRACT_STATUS_Refused_norm',
 'BUR_DAYS_CREDIT_mean',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'EXT_SOURCE_1',
 'DAYS_BIRTH']

In [14]:
# Polynomial Features
final = ft.poly(data=final, features=poly_features, degree=3)

# Modeling

In [15]:
final.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,FLAG_DAYS_EMPLOYED_ANOM,AMT_INCOME_PER_PERSON,CNT_APPR_INSTALL,PER_ANNUITY_INCOME,PER_ANNUITY_INCOME_P_PERSON,DAYS_BIRTHS_WHEN_LASTEST_CHANGE_REGIST,PER_DAYS_EMP_BIRTH,FlAG_AGE_GREATER_30,FLAG_EMPLOYMENT_GREATER_5,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,OCCUPATION_TYPE_Accountants,OCCUPATION

In [16]:
final.shape

(307511, 812)

__1. Modeling__

__EXPLANATION__
* `max_iter`: Maximum number of iterations taken for the solvers to converge
* `class_weight='balanced'`: The weights of each class will be automatically adjusted to address the imbalance in the number of samples between different classes in your dataset. Weight $w_i$ of class $i$ follows this formula:
$$w_i=\frac{n_{samples}}{n_{classes} * n_i}$$
* Notes:
    * $w_i$: weight of class $i$
    * $n_{samples}$: total number of samples
    * $n_{classes}$: the number of classes
    * $n_i$: the number of samples in class $i$

In [17]:
train, test = final[final['TARGET'].notnull()], final[final['TARGET'].isnull()]
X_train, y_train = train.iloc[:, 2:], train.iloc[:, 1]
X_test = test.iloc[:, 2:]

result = test[['SK_ID_CURR', 'TARGET']].copy()

logreg = LogisticRegression(max_iter=2000, class_weight='balanced')
logreg.fit(X_train, y_train)
result['TARGET'] = logreg.predict_proba(X_test).T[1]

In [18]:
result

,SK_ID_CURR,TARGET
246009,83659,0.77309
246010,174814,0.20571
246011,179486,0.65563
246012,57038,0.18233
246013,25672,0.41976
...,...,...
307506,150442,0.28850
307507,5217,0.16046
307508,260741,0.50280
307509,284794,0.21025


__2. Saving Result File__

In [19]:
result.set_index('SK_ID_CURR')
result.to_csv('submissions/submission.csv')